# Test GeneralizedEntailment Example

The knowledgebase under consideration:

In [1]:
from utils import *

kb = [Rule("", "A", 0.8), 
      Rule("", "B", 0.6), 
      Rule("A", "B", 0.9)]

Set up the constraint matrix

In [2]:
cm = constraints_matrix(kb)
print cm

[[ 0.2  0.2 -0.8 -0.8]
 [ 0.4 -0.6  0.4 -0.6]
 [ 0.1 -0.9  0.   0. ]]


Generate worlds for signatur

In [3]:
ws = worlds(signatur(kb))
print ws

[{'A': True, 'B': True}, {'A': True, 'B': False}, {'A': False, 'B': True}, {'A': False, 'B': False}]


First step is to generate the violation vector.

$$ \min_p ||Ap|| \\
    \text{s.t.} \sum_i p_i = 1$$
    
 Begin with the Euclidean-norm:

In [4]:
from cvxpy import *

ps = Variable(len(ws))
obj = Minimize(norm(cm*ps))
cons = [0 <= ps, ps <= 1, sum_entries(ps) == 1]
prob = Problem(obj, cons)

vioval2 = prob.solve()
print "Solution: ", vioval2
print "Probabilities:"
print ps.value

vio2 = cm*ps.value

print "Violation vector:"
print vio2

Solution:  0.0715859983384
Probabilities:
[[  6.42704641e-01]
 [  1.18861189e-01]
 [  4.39367699e-12]
 [  2.38434170e-01]]
Violation vector:
[[-0.03843417]
 [ 0.04270464]
 [-0.04270461]]


And the Manhatten-norm

In [5]:
obj = Minimize(norm(cm*ps, 1))
prob = Problem(obj, cons)

vioval1 = prob.solve()
print "Solution:", vioval1
print "Status:", prob.status
print "Probabilities:"
print ps.value

vio1 = cm*ps.value
print "Violation vector:"
print vio1

Solution: 0.119999999999
Status: optimal
Probabilities:
[[  6.36666998e-01]
 [  1.63333002e-01]
 [  3.80067874e-12]
 [  2.00000000e-01]]
Violation vector:
[[ -1.43246526e-11]
 [  3.66669981e-02]
 [ -8.33330019e-02]]


And the Maximum-norm

In [6]:
obj = Minimize(norm(cm*ps, "inf"))
prob = Problem(obj, cons)

viovalinf = prob.solve()
print "Solution:", viovalinf
print "Status:", prob.status
print "Probabilities:"
print ps.value

vioInf = cm*ps.value
print "Violation vector:"
print vioInf
print -1*vioInf

Solution: 0.0413793103138
Status: optimal
Probabilities:
[[  6.41379310e-01]
 [  1.17241379e-01]
 [ -3.53908138e-11]
 [  2.41379310e-01]]
Violation vector:
[[-0.04137931]
 [ 0.04137931]
 [-0.04137931]]
[[ 0.04137931]
 [-0.04137931]
 [ 0.04137931]]


Now query P(a) with the 2-Norm, 1-Norm, and Inf-Norm and the violation vector calculated by the 2-norm

In [7]:
vm = verifying_matrix(ws, kb[0])
fm = falsifying_matrix(ws, kb[0])
print vm
print fm

[1 1 0 0]
[0 0 1 1]


First the 2-Norm

In [14]:
x = Variable(len(ws))
t = Variable()

lower = Minimize(vm*x)
cons = [x >= 0, t >= 0,
        cm*x == t*vio2,
        sum_entries(x) == t,
        (vm + fm)*x == 1]
prob = Problem(lower, cons)

l = prob.solve()
print "Status lower:", prob.status

upper = Maximize(vm*x)
prob = Problem(upper, cons)

u = prob.solve()
print "Status upper:", prob.status

print (l, u)

# test util function
print "test query function"
print "P(A)"
l, u = query(kb[0], ws, kb)
print "%r, %r" % (l, u)
print "P(B)"
l, u = query(kb[1], ws, kb)
print "%r, %r" % (l, u)
print "P(B|A)"
l, u = query(kb[2], ws, kb)
print "%r, %r" % (l, u)

Status lower: optimal
Status upper: optimal
(0.7615658300418782, 0.7615658300449593)
test query function
P(A)
0.7615655714540518, 0.761565571465614
P(B)
0.6427047119612715, 0.6427047119702923
P(B|A)
0.8439256395511517, 0.8439256395573197


Now the 1-Norm

In [16]:
x = Variable(len(ws))
y = Variable(len(kb))
t = Variable()

cons = [x >= 0, y >= 0, t >= 0,
        -1*y <= cm*x, cm*x <= y,
        sum_entries(y) == t*vioval1,
        sum_entries(x) == t,
        (vm + fm)*x == 1]
lower = Minimize(vm*x)
prob = Problem(lower, cons)

l = prob.solve()
print "Status lower:", prob.status

upper = Maximize(vm*x)
prob = Problem(upper, cons)

u = prob.solve()
print "Status upper:", prob.status

print (l, u)

# test function
print "test query function"
print "P(A)"
l, u = query(kb[0], ws, kb, norm="1")
print "%r, %r" % (l, u)
print "P(B)"
l, u = query(kb[1], ws, kb, norm="1")
print "%r, %r" % (l, u)
print "P(B|A)"
l, u = query(kb[2], ws, kb, norm="1")
print "%r, %r" % (l, u)

Status lower: optimal
Status upper: optimal
(0.7999999996013246, 0.7999999999968094)
test query function
P(A)
0.7999999996087174, 0.7999999999963826
P(B)
0.6000000000001614, 0.7200000000062097
P(B|A)
0.7500000000068133, 0.9000000000801981


Now the Inf-Norm

In [17]:
x = Variable(len(ws))
t = Variable()

cons = [x >= 0, t >= 0,
        -t*viovalinf <= cm*x, cm*x <= t*viovalinf,
        sum_entries(x) == t,
        (vm+fm)*x == 1]
lower = Minimize(vm*x)
prob = Problem(lower,cons)

l = prob.solve()
print "Status lower:", prob.status

upper = Maximize(vm*x)
prob = Problem(upper, cons)

u = prob.solve()
print "Status upper:", prob.status

print (l,u)

# test function
print "test query function"
print "P(A)"
l, u = query(kb[0], ws, kb, norm="inf")
print "%r, %r" % (l, u)
print "P(B)"
l, u = query(kb[1], ws, kb, norm="inf")
print "%r, %r" % (l, u)
print "P(B|A)"
l, u = query(kb[2], ws, kb, norm="inf")
print "%r, %r" % (l, u)

Status lower: optimal
Status upper: optimal
(0.7586206896831538, 0.7586206896998027)
test query function
P(A)
0.7586206896287138, 0.758620689753735
P(B)
0.6413793102260176, 0.641379310366378
P(B|A)
0.8454545454275514, 0.845454545476464
